# Enformer-GTEx results

Explore Enformer predictions on GTEx variants

## Setup

In [1]:
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import plotnine as pn
from scipy.stats import ranksums
from datetime import datetime

%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['figure.dpi'] = 150

In [3]:
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [4]:
pn.theme_set(pn.theme_bw())
pn.theme_update(dpi=150)

In [5]:
# Files relevant to the GTEx analysis
enformer_run_name = 'enformer_gtexv8_elasticnet_cage_canonical_2000_500'
# enformer variant effect on GTEx variants
enformer_path = f'/s/project/promoter_prediction/kipoi_expression_prediction/veff/enformer.parquet/run={enformer_run_name}'
# GTEx variants
variant_path = '/s/project/rep/processed/training_results_v16/gtex_v8_old_dna/private_variants.parquet/rare_variants.vcf.parquet/**/*.parquet'
# abexp benchmark dataset
gtex_benchmark_with_annotation_path = "/s/project/rep/processed/training_results_v16/gtex_benchmark_with_annotation.parquet/*.parquet"

## Analysis

We are only loading ensembl canonical transcripts for this analysis. So there is 1 transcript per gene!

In [6]:
veff_transcript_ldf = (pl.scan_parquet(Path(enformer_path) / '*.parquet').
                       select(pl.col(['tissue', 'gene_id', 'transcript_id', 'transcript_start', 'transcript_end']), 
                              pl.col('strand').cast(pl.Enum(['-', '+'])),
                              pl.col(['chrom', 'variant_start', 'variant_end', 'ref', 'alt', 'veff_score', 'ref_score', 'alt_score'])).
                       rename({'gene_id': 'gene', 'transcript_id': 'transcript'}).
                       with_columns(pl.col('gene').str.replace(r'([^\.]+)\..+$', "${1}").alias('gene'),
                                    pl.col('transcript').str.replace(r'([^\.]+)\..+$', "${1}").alias('transcript')))


# It is possible that a gene comes multiple times (different versions)

In [7]:
variant_ldf = pl.scan_parquet(variant_path, hive_partitioning=True).select(['sampleId', 'chrom', 'start', 'end', 'ref', 'alt']).rename({'sampleId': 'individual',
                                                                                                                'start': 'variant_start',
                                                                                                                'end': 'variant_end'})

In [8]:
training_benchmark_ldf = (pl.scan_parquet(gtex_benchmark_with_annotation_path)
                          .select(['gene', 'individual', 'tissue', 'FDR', 
                                   'mu', 'zscore', 'l2fc'])
                          .unique()
                          .rename({'l2fc': 'l2fc_outrider'})
                          .with_columns((pl.when(pl.col('FDR') > 0.05)
                                         .then(pl.lit('normal'))
                                         .otherwise(
                                             pl.when(pl.col('zscore') > 0)
                                             .then(pl.lit('overexpressed'))
                                             .otherwise(
                                                 pl.when(pl.col('zscore') < 0)
                                                 .then(pl.lit('underexpressed'))
                                                 # this should never be the case
                                                 .otherwise(pl.lit('CHECK'))
                                             ))).cast(pl.Enum(['underexpressed', 'normal', 'overexpressed'])).alias('outlier_state')))

### What is the enformer variant-effect-score distribution around the TSS?

In [9]:
veff_transcript_ldf = veff_transcript_ldf. with_columns(tss_distance=(
    pl.when(pl.col('strand') == '+')
    .then(pl.col('variant_start') - pl.col('transcript_start'))
    .otherwise(pl.col('transcript_end') - pl.col('variant_start'))
)).with_columns(absolute_tss_distance=pl.col('tss_distance').abs(),
                absolute_veff_score=pl.col('veff_score').abs())

veff_transcript_ldf.select(upstream_tss=pl.col('tss_distance').min(), downstream_tss=pl.col('tss_distance').max()).collect()

upstream_tss,downstream_tss
i64,i64
-2000,500


In [10]:
upstream=2000
downstream=500

# filter out variants out of this range
veff_transcript_ldf = veff_transcript_ldf.filter((pl.col('tss_distance') >= -upstream) & (pl.col('tss_distance') <= downstream))

# join tss variants with individuals
veff_individual_ldf = (variant_ldf.join(veff_transcript_ldf, how='inner', on=['chrom', 'variant_start', 'variant_end', 'ref', 'alt']).
                              select([
                                  'individual', 'chrom', 'variant_start', 'variant_end', 'ref', 'alt',
                                  'gene', 'transcript', 'strand', 'tss_distance', 'absolute_tss_distance', 
                                  'veff_score', 'absolute_veff_score'
                                ])
                             )

In [11]:
# keep the variant with the signed maximum absolute enformer score for each individual
veff_individual_ldf = veff_individual_ldf.sort('absolute_veff_score', descending=True). \
    group_by(['individual', 'gene', 'chrom', 'strand']). \
    agg(pl.col(['variant_start', 'variant_end', 'ref', 'alt', 'transcript', 'tss_distance', 'veff_score']).first())

# join outrider with variants on individual
veff_outrider_ldf = (
    veff_individual_ldf
    .join(training_benchmark_ldf, how='inner', on=['individual', 'gene'])
    .select(['gene', 'tissue', 'individual', 'tss_distance', 'outlier_state', 
             'zscore', 'FDR', 'veff_score']))

In [12]:
df = veff_outrider_ldf.collect()

In [13]:
df.shape

(2607342, 8)

In [14]:
df.head()

gene,tissue,individual,tss_distance,outlier_state,zscore,FDR,veff_score
str,str,str,i64,enum,f32,f64,f32
"""ENSG00000128274""","""Skin - Sun Exposed (Lower leg)""","""GTEX-131XG""",181,"""normal""",-0.73513,1.0,0.020915
"""ENSG00000108518""","""Artery - Coronary""","""GTEX-14PN3""",-59,"""normal""",-0.690229,1.0,-0.075348
"""ENSG00000107140""","""Pancreas""","""GTEX-V1D1""",-1027,"""normal""",-0.533533,1.0,-0.006857
"""ENSG00000280433""","""Liver""","""GTEX-UPK5""",-1028,"""normal""",-0.259696,1.0,-0.000075
"""ENSG00000039319""","""Testis""","""GTEX-11DXY""",-424,"""normal""",-0.832431,1.0,-0.006894


In [15]:
df_bac = df

# tss_distance in bins
bin_size=50
cuts = list(range(-upstream + bin_size, downstream, bin_size))
cut_labels = [str(x) for x in [-upstream, *cuts]]
tss_distance_labels = {c: f'[{c}, {int(c) + bin_size})' for c in cut_labels}
df = (df.with_columns(tss_distance_bin=(pl.col('tss_distance').cut(cuts, labels=cut_labels))
                      .cast(pl.Enum(cut_labels)))
      .with_columns(is_underexpressed=(pl.col('outlier_state') == 'underexpressed')))

df = (df.with_columns(tss_distance_bin_label=pl.col("tss_distance_bin").replace_strict(tss_distance_labels), 
                                                      bin_size=pl.lit(50)))

In [16]:
df.head()

gene,tissue,individual,tss_distance,outlier_state,zscore,FDR,veff_score,tss_distance_bin,is_underexpressed,tss_distance_bin_label,bin_size
str,str,str,i64,enum,f32,f64,f32,enum,bool,str,i32
"""ENSG00000128274""","""Skin - Sun Exposed (Lower leg)""","""GTEX-131XG""",181,"""normal""",-0.73513,1.0,0.020915,"""150""",false,"""[150, 200)""",50
"""ENSG00000108518""","""Artery - Coronary""","""GTEX-14PN3""",-59,"""normal""",-0.690229,1.0,-0.075348,"""-100""",false,"""[-100, -50)""",50
"""ENSG00000107140""","""Pancreas""","""GTEX-V1D1""",-1027,"""normal""",-0.533533,1.0,-0.006857,"""-1050""",false,"""[-1050, -1000)""",50
"""ENSG00000280433""","""Liver""","""GTEX-UPK5""",-1028,"""normal""",-0.259696,1.0,-0.000075,"""-1050""",false,"""[-1050, -1000)""",50
"""ENSG00000039319""","""Testis""","""GTEX-11DXY""",-424,"""normal""",-0.832431,1.0,-0.006894,"""-450""",false,"""[-450, -400)""",50


In [17]:
# set new bins
# new_bins = [(-2000, -500),
#             *[(i, i + 100) for i in range(-500, -100, 100)],
#             (-100, -50),
#             (-50, 0),
#             (0, 50),
#             (50, 100),
#             *[(i, i + 100) for i in range(100, 500, 100)],]
new_bins = [(-2000, -1500),
            (-1500, -1000),
            (-1000, -500),
            *[(i, i + 100) for i in range(-500, -100, 100)],
            (-100, -50),
            (-50, 0),
            (0, 50),
            (50, 100),
            *[(i, i + 100) for i in range(100, 500, 100)],
           ]
new_bin_labels  = [f'[{start}, {stop})' for start, stop in new_bins]

df_bac = df
for start, stop in new_bins:
    df = df.with_columns(tss_distance_bin_label=(pl.when((pl.col('tss_distance_bin').cast(pl.Int16) >= start) & (pl.col('tss_distance_bin').cast(pl.Int16) <= stop))
                                                 .then(pl.lit(f'[{start}, {stop})'))                                  
                                                 .otherwise(pl.col('tss_distance_bin_label'))),
                         bin_size = (pl.when((pl.col('tss_distance_bin').cast(pl.Int16) >= start) & (pl.col('tss_distance_bin').cast(pl.Int16) < stop))
                                     .then(pl.lit(stop - start))
                                     .otherwise(pl.col('bin_size'))))

df = df.with_columns(tss_distance_bin_label=pl.col('tss_distance_bin_label').cast(pl.Enum(new_bin_labels)))

In [18]:
df.head()

gene,tissue,individual,tss_distance,outlier_state,zscore,FDR,veff_score,tss_distance_bin,is_underexpressed,tss_distance_bin_label,bin_size
str,str,str,i64,enum,f32,f64,f32,enum,bool,enum,i32
"""ENSG00000128274""","""Skin - Sun Exposed (Lower leg)""","""GTEX-131XG""",181,"""normal""",-0.73513,1.0,0.020915,"""150""",false,"""[100, 200)""",100
"""ENSG00000108518""","""Artery - Coronary""","""GTEX-14PN3""",-59,"""normal""",-0.690229,1.0,-0.075348,"""-100""",false,"""[-100, -50)""",50
"""ENSG00000107140""","""Pancreas""","""GTEX-V1D1""",-1027,"""normal""",-0.533533,1.0,-0.006857,"""-1050""",false,"""[-1500, -1000)""",500
"""ENSG00000280433""","""Liver""","""GTEX-UPK5""",-1028,"""normal""",-0.259696,1.0,-0.000075,"""-1050""",false,"""[-1500, -1000)""",500
"""ENSG00000039319""","""Testis""","""GTEX-11DXY""",-424,"""normal""",-0.832431,1.0,-0.006894,"""-450""",false,"""[-500, -400)""",100


In [19]:
# calculate counts per bins
bin_count_df = (df.group_by(['outlier_state', 'tss_distance_bin_label', 'bin_size'])
                .agg((pl.len()).alias('count')))

# # calculate mean of each bin and then sum the means in each outlier state
totals_df = (bin_count_df.group_by('outlier_state').agg(pl.sum('count').alias('total_count')))
# # normalize each count by the mean calculated above
enrichment_df = (bin_count_df.join(totals_df, on='outlier_state')
                 .with_columns((pl.col('count') / pl.col('total_count')).alias('enrichment')))
ci_low, ci_high = sm.stats.proportion_confint(enrichment_df["count"], enrichment_df["total_count"])
enrichment_df = enrichment_df.with_columns(pl.Series(ci_low).alias('ci_low'), pl.Series(ci_high).alias('ci_high'))
# normalize by bin size
enrichment_df = enrichment_df.with_columns(enrichment = pl.col('enrichment') / pl.col('bin_size'),
                                           ci_low = pl.col('ci_low') / pl.col('bin_size'),
                                           ci_high = pl.col('ci_high') / pl.col('bin_size'))

In [20]:
enrichment_df

outlier_state,tss_distance_bin_label,bin_size,count,total_count,enrichment,ci_low,ci_high
enum,enum,i32,u32,u32,f64,f64,f64
"""normal""","""[-2000, -1500)""",500,449063,2605370,0.000345,0.000344,0.000346
"""underexpressed""","""[-500, -400)""",100,17,1238,0.000137,0.000072,0.000202
"""underexpressed""","""[100, 200)""",100,134,1238,0.001082,0.000909,0.001255
"""normal""","""[-400, -300)""",100,110099,2605370,0.000423,0.00042,0.000425
"""underexpressed""","""[-400, -300)""",100,61,1238,0.000493,0.000372,0.000613
…,…,…,…,…,…,…,…
"""normal""","""[100, 200)""",100,106783,2605370,0.00041,0.000407,0.000412
"""overexpressed""","""[-500, -400)""",100,23,734,0.000313,0.000187,0.000439
"""overexpressed""","""[100, 200)""",100,69,734,0.00094,0.000729,0.001151


In [21]:
totals_df

outlier_state,total_count
enum,u32
"""underexpressed""",1238
"""normal""",2605370
"""overexpressed""",734


In [22]:
base_path = Path('.')
df.write_parquet(base_path / 'veff_bin.parquet', use_pyarrow=True)
enrichment_df.write_parquet(base_path / 'promoter_enrichment.parquet', use_pyarrow=True)
base_path

PosixPath('.')